In [1]:
import pandas as pd
import numpy as np

In [5]:
data = {'Name': ['A', 'B', 'C'], 'Start_time': ['2017-12-15', '2017-12-15', '2017-12-15'], 'End_time': ['2018-01-17', '2017-12-22', '2018-05-22']}
df = pd.DataFrame(data)

In [38]:
df = df.astype({'Start_time': 'datetime64[ns]', 'End_time': 'datetime64[ns]'})

In [40]:
df.dtypes

Name                  object
Start_time    datetime64[ns]
End_time      datetime64[ns]
dtype: object

In [44]:
df

,Name,Start_time,End_time
0,A,2017-12-15,2018-01-17
1,B,2017-12-15,2017-12-22
2,C,2017-12-15,2018-05-22


In [47]:
from pandas.tseries.offsets import *
df['LastDay'] = df['Start_time'] + MonthEnd()

In [48]:
df

,Name,Start_time,End_time,LastDay
0,A,2017-12-15,2018-01-17,2017-12-31
1,B,2017-12-15,2017-12-22,2017-12-31
2,C,2017-12-15,2018-05-22,2017-12-31


In [50]:
df.End_time <= df.LastDay 

0    False
1     True
2    False
dtype: bool

In [54]:
dList = []
vList = []
d = {}

def buildDict(row):   
    for x in pd.date_range(row["Start_time"],row["End_time"], freq='MS'):  #build a range for each row
        dList.append(x)  #date list
        vList.append(row["Name"]) #value list 

df.apply(buildDict,axis=1) #each row in df is passed to buildDict

#this d will be used to create our new frame
d["date"] = dList
d["value"] = vList

In [56]:
pd.DataFrame(d)

,date,value
0,2018-01-01,A
1,2018-01-01,C
2,2018-02-01,C
3,2018-03-01,C
4,2018-04-01,C
5,2018-05-01,C
